# Trying to use time features

In [1]:
import pandas as pd

df = pd.read_csv('Whole_dataset1.txt')

df = df.sort_values(by='Date')

import gc; gc.collect()

53

## Creating a new feature 'timestamp'

In [2]:
date = df.Date.tolist()
timestamp = [int(''.join(i.split('-'))) for i in date]
df['timestamp'] = timestamp; del timestamp
del date

## Doing joins

In [3]:
df = df.drop(axis=1,labels=['Date'])

ddf = pd.concat([pd.read_pickle('info_1.pkl'),
          pd.read_pickle('info_2.pkl'),
          pd.read_pickle('info_3.pkl')])

df = df.join(ddf.set_index('MovieID'),on='MovieID')
del ddf

ddf = pd.concat([pd.read_pickle('movie_info1.pkl'),
          pd.read_pickle('movie_info2.pkl'),
          pd.read_pickle('movie_info3.pkl'),
                pd.read_pickle('movie_info4.pkl'),
                pd.read_pickle('movie_info5.pkl')])

df = df.join(ddf.set_index('CustomerID'),on='CustomerID')
del ddf

ddf = pd.read_pickle('modified_movie_title_year.pkl')

df = df.join(ddf.set_index('MovieID'),on='MovieID')
del ddf

In [ ]:
from sklearn.metrics import mean_squared_error
import math
rmse = []

### Logistic regression with simple time features

In [ ]:
'''Printing a sample from the traindata to see the features that have been used'''
df.head(3)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df.shape[0]*.7)
reg.fit(df.drop(axis=1,labels=['Score'])[:l],df.Score[:l])
hypothesis = reg.predict(df.drop(axis=1,labels=['Score'])[l:])

In [ ]:
e = math.sqrt(mean_squared_error(df.Score[l:],hypothesis))
e

In [ ]:
rmse.append(e)

### Using SGD Regressor

In [ ]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor()
l = int(df.shape[0]*.7)
reg.fit(df.drop(axis=1,labels=['Score'])[:l],df.Score[:l])
hypothesis = reg.predict(df.drop(axis=1,labels=['Score'])[l:])

In [ ]:
e = math.sqrt(mean_squared_error(df.Score[l:],hypothesis))
e

In [ ]:
print (hypothesis[:10])

    Something is going wrong with SGD Regressor

    There isnt't much increase in the performance. Let's consider only the data from 2004 and onwards

In [ ]:
df_2004 = df[df.timestamp > 20040000]
df_2004.shape

In [ ]:
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df_2004.shape[0]*.7)
reg.fit(df_2004.drop(axis=1,labels=['Score'])[:l],df_2004.Score[:l])
hypothesis = reg.predict(df_2004.drop(axis=1,labels=['Score'])[l:])

In [ ]:
e = math.sqrt(mean_squared_error(df_2004.Score[l:],hypothesis))
del df_2004
e

    Let's see the same on only 2005 data

In [ ]:
df_2005 = df[df.timestamp > 20050000]
df_2005.shape

In [ ]:
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df_2005.shape[0]*.7)
reg.fit(df_2005.drop(axis=1,labels=['Score'])[:l],df_2005.Score[:l])
hypothesis = reg.predict(df_2005.drop(axis=1,labels=['Score'])[l:])

In [ ]:
e = math.sqrt(mean_squared_error(df_2005.Score[l:],hypothesis))
del df_2005
e

In [ ]:
del df

# Including vectors from MF 

In [4]:
import pickle
with open('Matrix10_UM.pkl','rb') as fp:
    user_mat, mov_mat = pickle.load(fp)
fp.close()
user_mat.shape, mov_mat.shape

((470758, 10), (10, 4499))

In [5]:
cus = sorted(list(set(df['CustomerID'])))
cus = pd.DataFrame(data = cus)
user_mat = pd.DataFrame(data = user_mat,columns=['U1','U2','U3','U4','U5','U6','U7','U8','U9','U10'])
user_mat['CustomerID'] = cus
del cus
user_mat.head(2)

,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,CustomerID
0,0.247516,1.530273,0.667864,0.157511,0.000000,0.054146,0.081579,0.000000,0.14167,0.220464,6
1,0.164473,2.604064,0.633440,0.452483,0.391106,0.319762,0.007162,0.269236,0.18859,0.193465,7


In [6]:
mov_mat = mov_mat.T
mov_mat = pd.DataFrame(data = mov_mat,\
columns=['M1','M2','M3','M4','M5','M6','M7','M8','M9','M10'])
mov = list(range(1,4500))
mov_mat['MovieID'] = mov
del mov
mov_mat.head(2)

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,MovieID
0,0.012890,0.000000,0.000596,0.016911,0.002714,0.0,0.00000,0.0,0.000000,0.000694,1
1,0.000117,0.000472,0.000000,0.000000,0.000000,0.0,0.00397,0.0,0.001019,0.001949,2


### Table joins

In [7]:
df = df.join(user_mat.set_index('CustomerID'),on='CustomerID')
del user_mat
df = df.join(mov_mat.set_index('MovieID'),on='MovieID')
del mov_mat
Y = df.Score.tolist()
df = df.drop(axis=1,labels=['Score','CustomerID','MovieID'])
df.head(2)

,timestamp,NumberOfRating,MeanRating,StdDev,Cust_mean,cust_stddev,freq_of_rating,Year_of_release,U1,U2,...,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10
9056171,19991111,113377,3.971317,0.893782,3.364807,0.958341,233,19870101,0.558849,0.0,...,0.000000,0.55414,0.463257,1.385780,0.000000,4.994782,0.0,0.0,5.884801,0.0
14892677,19991111,22415,3.439929,1.099609,3.364807,0.958341,233,19990101,0.558849,0.0,...,0.130603,0.00000,0.000000,0.206944,1.469368,1.461222,0.0,0.0,0.119141,0.0


In [10]:
l = int(df.shape[0]*.7)
ts_df = df[l:]; df = df[:l] 
ts_Y = Y[l:]; Y = Y[:l]

In [11]:
import gc
gc.collect()

0

## Linear Regression

In [12]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
reg.fit(df,Y)
hypothesis = reg.predict(ts_df)

In [13]:
from sklearn.metrics import mean_squared_error
import math
m = math.sqrt(mean_squared_error(ts_Y,hypothesis))
m

0.9045219722876191

In [ ]:
print(test)

In [14]:
df.columns

Index(['timestamp', 'NumberOfRating', 'MeanRating', 'StdDev', 'Cust_mean',
       'cust_stddev', 'freq_of_rating', 'Year_of_release', 'U1', 'U2', 'U3',
       'U4', 'U5', 'U6', 'U7', 'U8', 'U9', 'U10', 'M1', 'M2', 'M3', 'M4', 'M5',
       'M6', 'M7', 'M8', 'M9', 'M10'],
      dtype='object')